In [ ]:
# === !!!成品!!! ===

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
# from webdriver_manager.chrome import ChromeDriverManager
import time
import os, json


#滾輪
def scroll_to_bottom(driver, step=800):
    current = 0
    while True:
        driver.execute_script(f"window.scrollTo(0, {current});")
        current += step
        time.sleep(0.05)
        # 每次往下滾一段，直到超過目前頁面的總高度就停
        if current >= driver.execute_script("return document.body.scrollHeight"):
            break

#嘗試典籍下一頁並抓取資料
def scrape_current_page(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR,".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")
        )
    )

    scroll_to_bottom(driver)


    razer_lists = driver.find_elements(By.CSS_SELECTOR, ".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")

    max_wait = 5
    start = time.time()
    while True:
        imgs = driver.find_elements(By.CSS_SELECTOR, ".c-prodInfoV2__head img")
        
        if not imgs:
            if time.time() - start > max_wait:
                break
            time.sleep(0.2)
            continue

        last_src = imgs[-1].get_attribute("src") or ""
        if not last_src.endswith(".svg"):
            break
        
        if time.time() -start > max_wait:
            break
        
        time.sleep(0.2)

    all_razer = []
    for razer_list in razer_lists:
            title = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title")
            razer_name = title.get_attribute("title")
            razer_price = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__priceBar .c-prodInfoV2__price").text
            link = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__link.gtmClickV2")
            razer_link = link.get_attribute("href")
            if razer_link.startswith("/"):
                razer_link = "https://24h.pchome.com.tw" + razer_link
            img = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__head img")
            razer_imgsrc = img.get_attribute("src")
            
            all_razer.append({
                "name": razer_name,
                "price": razer_price,
                "link": razer_link,
                "img_src": razer_imgsrc
            })

    return all_razer

# ①建立 driver

def crawl_pchome(keyword:str ,pages: int = 18):
    service = Service()  # 如果 chromedriver 在 PATH 裡，這樣就可以；不行就要給 executable_path
    driver = webdriver.Chrome(service=service)
    all_razer = []

    url = f"https://24h.pchome.com.tw/search/?q={keyword}"
    driver.get(url)



    for page in range(1, pages+1):
        print(f"=== 查詢商品: {keyword} : 現在是第 {page} 頁 ===")        
        items = scrape_current_page(driver)
        all_razer.extend(items)

        if page == pages:
            break
        
        try:
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, ".c-pagination__button.is-next")
                )
            )
            next_btn.click()
            time.sleep(2)
        except TimeoutException:
            print("找不到下一頁按鈕，提前結束。")
            break
    driver.quit()

    filename = f"{keyword}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_razer, f, ensure_ascii=False, indent=2)

    print(f"{keyword} 總共抓到 {len(all_razer)} 筆，已輸出到 {filename}")
    return all_razer

# 呼叫 function crawl_pchome
crawl_pchome("razer")
        

=== 查詢商品: razer : 現在是第 1 頁 ===
=== 查詢商品: razer : 現在是第 2 頁 ===
=== 查詢商品: razer : 現在是第 3 頁 ===
=== 查詢商品: razer : 現在是第 4 頁 ===
=== 查詢商品: razer : 現在是第 5 頁 ===
=== 查詢商品: razer : 現在是第 6 頁 ===
=== 查詢商品: razer : 現在是第 7 頁 ===
=== 查詢商品: razer : 現在是第 8 頁 ===
=== 查詢商品: razer : 現在是第 9 頁 ===
=== 查詢商品: razer : 現在是第 10 頁 ===
=== 查詢商品: razer : 現在是第 11 頁 ===
=== 查詢商品: razer : 現在是第 12 頁 ===
=== 查詢商品: razer : 現在是第 13 頁 ===
=== 查詢商品: razer : 現在是第 14 頁 ===
=== 查詢商品: razer : 現在是第 15 頁 ===
=== 查詢商品: razer : 現在是第 16 頁 ===
=== 查詢商品: razer : 現在是第 17 頁 ===
=== 查詢商品: razer : 現在是第 18 頁 ===
razer 總共抓到 713 筆，已輸出到 razer.json


[{'name': 'RAZER 雷蛇 DeathAdder V3 PRO 煉獄蝰蛇 超輕量無線人體工學滑鼠',
  'price': '$3,290',
  'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900GHTKW',
  'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GHTKW/000001_1762272160.jpg?width=480'},
 {'name': 'RAZER 雷蛇 Basilisk V3 X HyperSpeed  巴塞利斯蛇 V3 X 速度版 無線電競滑鼠',
  'price': '$1,888',
  'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900GXSL6',
  'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GXSL6/000001_1762272172.jpg?width=480'},
 {'name': 'RAZER 雷蛇 DeathAdder Essential 蝰蛇標準版 電競滑鼠',
  'price': '$518',
  'link': 'https://24h.pchome.com.tw/prod/DCBF00-A900BRF9M',
  'img_src': 'https://img.pchome.com.tw/cs/items/DCBF00A900BRF9M/000001_1762272150.jpg?width=480'},
 {'name': 'RAZER 雷蛇 Pro Click Mini無線滑鼠',
  'price': '$1,499',
  'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900HEH1W',
  'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900HEH1W/000001_1762272179.jpg?width=480'},
 {'name': 'RAZER 雷蛇 DeathAdder V2 X 煉獄奎蛇速度版無線電競滑鼠',


In [67]:
razer_lists = driver.find_elements(By.CSS_SELECTOR, ".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")

all_razer = []
for razer_list in razer_lists:
    try:
            title = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title")
            razer_name = title.get_attribute("title")
            razer_price = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__priceBar .c-prodInfoV2__price").text
            link = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__link.gtmClickV2")
            razer_link = link.get_attribute("href")
            if razer_link.startswith("/"):
                razer_link = "https://24h.pchome.com.tw" + razer_link
            img = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__flex img")
            razer_imgsrc = img.get_attribute("src")
            
            all_razer.append({
                "name": razer_name,
                "price": razer_price,
                "link": razer_link,
                "img_src": razer_imgsrc
            })
    except Exception as e:
        print("某個商品解析失敗：", e)
        continue

print(all_razer)
print("成功解析商品數量：", len(all_razer))



[{'name': 'RAZER 雷蛇 DeathAdder V3 PRO 煉獄蝰蛇 超輕量無線人體工學滑鼠', 'price': '$2,990', 'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900GHTKW', 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GHTKW/000001_1762272160.jpg?width=480'}, {'name': 'RAZER 雷蛇 DeathAdder V4 PRO 煉獄奎蛇超輕量無線人體工學電競滑鼠', 'price': '$5,190', 'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900IZY50', 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900IZY50/000007_1752489465.jpg?width=480'}, {'name': 'RAZER 雷蛇 DeathAdder V2 X 煉獄奎蛇速度版無線電競滑鼠', 'price': '$999', 'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900GLFS7', 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GLFS7/000007_1731896492.jpg?width=480'}, {'name': 'RAZER 雷蛇 Basilisk V3 X HyperSpeed  巴塞利斯蛇 V3 X 速度版 無線電競滑鼠', 'price': '$1,888', 'link': 'https://24h.pchome.com.tw/prod/DCBF01-A900GXSL6', 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GXSL6/000007_1731896503.jpg?width=480'}, {'name': 'RAZER 雷蛇 Viper V3 PRO 毒蝰超輕量無線滑鼠', 'price': '$4,288', '

In [ ]:
#嘗試典籍下一頁並抓取資料
def scrape_current_page(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            By.CSS_SELECTOR,".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item"
        )
    )
    razer_lists = driver.find_elements(By.CSS_SELECTOR, ".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")

    driver.find_element(By.CSS_SELECTOR, ".c-pagination__button.is-next").click()


    all_razer = []
    for razer_list in razer_lists:
            title = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title")
            razer_name = title.get_attribute("title")
            razer_price = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__priceBar .c-prodInfoV2__price")
            link = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__link.gtmClickV2")
            razer_link = link.get_attribute("href")
            if razer_link.startswith("/"):
                razer_link = "https://24h.pchome.com.tw" + razer_link
            img = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__flex img")
            razer_imgsrc = img.get_attribute("src")
            
            all_razer.append({
                "name": razer_name,
                "price": razer_price,
                "link": razer_link,
                "img_src": razer_imgsrc
            })

    return all_razer


[{'name': 'RAZER 雷蛇 DeathAdder V3 PRO 煉獄蝰蛇 超輕量無線人體工學滑鼠', 'price': '$2,990', 'img': <selenium.webdriver.remote.webelement.WebElement (session="e312e9bb30b05b8398eb78e8ef4e9d3b", element="f.D7C3AD80E8CE62DD51670E8D5ED2169D.d.82495C9ABC08C5F180BE54C2AB5EBFD4.e.93")>, 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900GHTKW/000001_1762272160.jpg?width=480'}, {'name': 'RAZER 雷蛇 DeathAdder V4 PRO 煉獄奎蛇超輕量無線人體工學電競滑鼠', 'price': '$5,190', 'img': <selenium.webdriver.remote.webelement.WebElement (session="e312e9bb30b05b8398eb78e8ef4e9d3b", element="f.D7C3AD80E8CE62DD51670E8D5ED2169D.d.82495C9ABC08C5F180BE54C2AB5EBFD4.e.96")>, 'img_src': 'https://img.pchome.com.tw/cs/items/DCBF01A900IZY50/000007_1752489465.jpg?width=480'}, {'name': 'RAZER 雷蛇 DeathAdder V2 X 煉獄奎蛇速度版無線電競滑鼠', 'price': '$999', 'img': <selenium.webdriver.remote.webelement.WebElement (session="e312e9bb30b05b8398eb78e8ef4e9d3b", element="f.D7C3AD80E8CE62DD51670E8D5ED2169D.d.82495C9ABC08C5F180BE54C2AB5EBFD4.e.99")>, 'img_src': 'https

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
# from webdriver_manager.chrome import ChromeDriverManager
import time

#嘗試典籍下一頁並抓取資料
def scrape_current_page(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR,".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")
        )
    )
    razer_lists = driver.find_elements(By.CSS_SELECTOR, ".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item")

    all_razer = []
    for razer_list in razer_lists:
            title = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title")
            razer_name = title.get_attribute("title")
            razer_price = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__priceBar .c-prodInfoV2__price").text
            link = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__link.gtmClickV2")
            razer_link = link.get_attribute("href")
            if razer_link.startswith("/"):
                razer_link = "https://24h.pchome.com.tw" + razer_link
            img = razer_list.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__flex img")
            razer_imgsrc = img.get_attribute("src")
            
            all_razer.append({
                "name": razer_name,
                "price": razer_price,
                "link": razer_link,
                "img_src": razer_imgsrc
            })

    return all_razer

# ① 建立 driver（這段很重要！）
service = Service()  # 如果 chromedriver 在 PATH 裡，這樣就可以；不行就要給 executable_path
driver = webdriver.Chrome(service=service)
all_razer = []

# 先進到【第 1 頁】搜尋結果
driver.get("https://24h.pchome.com.tw/search/?q=razer")



for page in range(1, 18+1):
    print(f"=== 現在是第 {page} 頁 ===")
    
    items = scrape_current_page(driver)
    print(f"第 {page} 頁，抓到 {len(items)} 筆")
    all_razer.extend(items)

    if page == 18:
        break
    
    try:
        next_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, ".c-pagination__button.is-next")
            )
        )
        next_btn.click()
        time.sleep(2)
    except TimeoutException:
        print("找不到下一頁按鈕，提前結束。")
        break

print("總共抓到商品筆數: ", len(all_razer))
        

=== 現在是第 1 頁 ===
第 1 頁，抓到 40 筆
=== 現在是第 2 頁 ===
第 2 頁，抓到 40 筆
=== 現在是第 3 頁 ===
第 3 頁，抓到 40 筆
=== 現在是第 4 頁 ===
第 4 頁，抓到 40 筆
=== 現在是第 5 頁 ===
第 5 頁，抓到 40 筆
=== 現在是第 6 頁 ===
第 6 頁，抓到 40 筆
=== 現在是第 7 頁 ===
第 7 頁，抓到 40 筆
=== 現在是第 8 頁 ===
第 8 頁，抓到 40 筆
=== 現在是第 9 頁 ===
第 9 頁，抓到 40 筆
=== 現在是第 10 頁 ===
第 10 頁，抓到 40 筆
=== 現在是第 11 頁 ===
第 11 頁，抓到 40 筆
=== 現在是第 12 頁 ===
第 12 頁，抓到 40 筆
=== 現在是第 13 頁 ===
第 13 頁，抓到 40 筆
=== 現在是第 14 頁 ===
第 14 頁，抓到 40 筆
=== 現在是第 15 頁 ===
第 15 頁，抓到 40 筆
=== 現在是第 16 頁 ===
第 16 頁，抓到 40 筆
=== 現在是第 17 頁 ===
第 17 頁，抓到 40 筆
=== 現在是第 18 頁 ===
第 18 頁，抓到 33 筆
總共抓到商品筆數:  713


In [45]:
#點擊下一頁
driver.find_element(By.CSS_SELECTOR, ".c-pagination__button.is-next").click()

In [ ]:
def scrape_current_page(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            By.CSS_SELECTOR,".c-listInfoGrid.c-listInfoGrid--gridCard ul li.c-listInfoGrid__item"
        )
    )

In [ ]:
# driver.find_element(By.CLASS_NAME, "c-search__input").send_keys("razer")

In [ ]:
# driver.find_element(By.CSS_SELECTOR, ".btn.btn--xs.gtmClickV2 span.btn__circular.btn__circular--white").click()